In [1]:
import pandas as pd
import os
import glob

In [3]:
data_path = '../data/raw/official_air_pollution_IN/'
all_sites = []
for loc in os.listdir(data_path):
    loc_path = os.path.join(data_path, loc)
    files = glob.glob(os.path.join(loc_path, '*.xlsx'))
    data = []
    for f in files:
        name = os.path.basename(f)[:-4]
        result = pd.read_excel(f, header=None, na_values=['None'], )
        meta = result.iloc[:17]
        values = result.iloc[17:]
        values.columns = list(result.iloc[16])
        values = values.rename(columns={'From Date': 'date'}).drop('To Date', axis=1)
        values['date'] = pd.to_datetime(values['date'], dayfirst=True)
        values = values.set_index('date', drop=True)
        values = values.convert_dtypes()
        data.append(values)

    combined = pd.concat(data).groupby(level=0).mean()
    # combined.to_csv('../data/testgitignored.csv')
    combined['location'] = loc
    combined = combined.set_index([combined.index, 'location'])
    all_sites.append(combined)

In [11]:
final = pd.concat(all_sites).sort_index()
final.columns = [f + '_govt' for f in final.columns]
final = final.astype('float64')
final.to_csv('../data/cleaned/government.csv')
final.to_pickle('../data/cleaned/government.pkl')
final

NO2_govt   SO2_govt   CO_govt  PM2.5_govt  NH3_govt
date       location                                                       
2018-01-01 Agartala         NaN        NaN       NaN         NaN       NaN
           Agra       50.290000   9.970000       NaN  341.930000       NaN
           Aizawl           NaN        NaN       NaN         NaN       NaN
           Amritsar   30.430000   2.780000       NaN         NaN       NaN
           Asansol          NaN        NaN       NaN         NaN       NaN
...                         ...        ...       ...         ...       ...
2022-08-24 Ghaziabad  14.330000  15.820000       NaN   11.633333       NaN
           Ludhiana    6.060000   8.600000  0.590000   38.490000       NaN
           Mangalore  10.350000   6.030000  1.030000         NaN       NaN
           Mumbai     10.104286   9.385714  0.658125   10.061250       NaN
2022-08-25 Chennai    16.090000  10.333750       NaN   20.497143       NaN

[91560 rows x 5 columns]

In [12]:
final.dtypes

NO2_govt      float64
SO2_govt      float64
CO_govt       float64
PM2.5_govt    float64
NH3_govt      float64
dtype: object

In [10]:
final['NO2_govt'].astype('float64')

date        location 
2018-01-01  Agartala           NaN
            Agra         50.290000
            Aizawl             NaN
            Amritsar     30.430000
            Asansol            NaN
                           ...    
2022-08-24  Ghaziabad    14.330000
            Ludhiana      6.060000
            Mangalore    10.350000
            Mumbai       10.104286
2022-08-25  Chennai      16.090000
Name: NO2_govt, Length: 91560, dtype: float64